In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [12]:
#Coloque o caminho em que está a pasta 'cg-proc' que foi clonada do repositório - NÃO EXCLUA OS OUTROS CAMINHOS, APENAS COMENTE.
master_path = "C:/Users/bruno/Desktop/Coding/GitHub/"
#master_path = "home/ken/KR/USP/7S/CG"

In [13]:
# Dicionário para facilitar a refêrencia das imagens antes do threshold
image_base_path = "{}cg-proc/img/".format(master_path)

images_dict = {}

for i in range(1, 166):
    image_name = f"{i}.jpeg"  # Nome da imagem
    image_path = image_base_path + image_name  
    images_dict[f"{i}"] = image_path  # 1, 2 ..., 3

In [14]:
#Funções de auxilio para plicar as transformações na imagem

def contrasteBrilho(image,alpha,beta):
    return cv2.convertScaleAbs(image, alpha, beta)

def rgb_to_gray(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray_image

def threshold(image, threshold_value):
    _, binarized_image = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
    return binarized_image

def adapt_threshold(image):
    binarized_image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 4)
    return binarized_image

def OTSU_threshold(image):
    ret, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return thresh

def negative(image):
    negative_image = cv2.bitwise_not(image)
    return negative_image

In [15]:
def fill_holes(image):
    # Imagem para o preenchimento é uma cópia
    filled_image = image.copy()
    
    # Outra imagem com fundo branco, para modificação
    h, w = image.shape
    white_image = np.ones((h, w), np.uint8) * 255
    
    # Flood fill a partir de um ponto fora da imagem (por exemplo, (0, 0))
    flood_filled = image.copy()
    mask = np.zeros((h + 2, w + 2), np.uint8)  # Máscara sendo 2 pixels maior que a imagem
    cv2.floodFill(flood_filled, mask, (0, 0), 255)
    
    # Inverter a imagem de flood fill
    flood_filled_inv = cv2.bitwise_not(flood_filled)
    
    # Obter somente os buracos
    holes = cv2.bitwise_and(white_image, white_image, mask=flood_filled_inv)
    
    # Adicionar os buracos de volta à imagem original
    filled_image = cv2.bitwise_or(image, holes)
    
    return filled_image

In [16]:
def detect_circles(image):
    # Parametros da Transformada de Hough
    circles = cv2.HoughCircles(
        image,
        cv2.HOUGH_GRADIENT,
        dp=1,
        minDist=30,
        param1=27,
        param2=26,
        minRadius=17,
        maxRadius=100
    )
    return circles

# Função para desenhar círculos detectados na imagem
def draw_circles(image, circles, save_path):
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            # Desenhar o contorno do círculo com borda grossa e vermelha
            cv2.circle(image, (i[0], i[1]), i[2], (0, 0, 255), 5)
            # Desenhar o centro do círculo com borda grossa e vermelha
            cv2.circle(image, (i[0], i[1]), 2, (0, 0, 255), 3)
        cv2.imwrite(save_path, image)

In [17]:
#Ajuste de brilho e contraste (TESTE)
for name, img_path in images_dict.items():
    image = cv2.imread(img_path)
    brilho = 100
    contraste = 1.5
    image = contrasteBrilho(image,brilho,contraste)

In [18]:
# Primeira etapa: threshold com o método de OTSU
# Método de OTSU: um dos mais populares algoritmos de threshold que determina o limiar ótimo
for name, img_path in images_dict.items():
    image = cv2.imread(img_path)
    gray_image = rgb_to_gray(image)
    binarized_image = OTSU_threshold(gray_image)
    cv2.imwrite("{}cg-proc/img/edited/t_hold_OTSU_{}.jpeg".format(master_path, name), binarized_image)

In [19]:
# Dicionário para facilitar a refêrencia das imagens após o threshold

edited_image_base_path = "{}cg-proc/img/edited/t_hold_OTSU_".format(master_path)

images_dict_edited = {}

for i in range(1, 166):
    image_name = f"{i}.jpeg"  # Nome da imagem editada
    image_path = edited_image_base_path + image_name  
    images_dict_edited[f"{i}_"] = image_path  

In [20]:
for name, img_path in images_dict_edited.items():
    # Ler a imagem em tons de cinza e aplicar algumas correções
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    brilho = 100
    contraste = 1.2
    image = contrasteBrilho(image, brilho, contraste)

    # Aplicar filtro de mediana
    median_blurred = cv2.medianBlur(image, 7)

    # Inverter as cores da imagem
    negative_image = negative(median_blurred)

    # Preencher buracos na imagem e adicionar a imagem na pasta EDITED
    filled_image = fill_holes(negative_image)
    cv2.imwrite("{}cg-proc/img/edited/{}_FILLED.jpeg".format(master_path, name), filled_image)
    
    # Remover ruído usando abertura 
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(filled_image, cv2.MORPH_OPEN, kernel, iterations=2)
    
    # Área de fundo seguro
    sure_bg = cv2.dilate(opening, kernel, iterations=3)
    
    # Área de primeiro plano seguro
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    #cv2.imwrite("{}cg-proc/img/edited/{}_DIST.jpeg".format(master_path, name), dist_transform)
    ret, sure_fg = cv2.threshold(dist_transform, 0.6 * dist_transform.max(), 255, 0)
    
    # Área da região desconhecida
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    
    # Rotulagem de marcadores
    ret, markers = cv2.connectedComponents(sure_fg)
    
    # Adicionar um a todos os rótulos para que o fundo seguro não seja 0, mas 1
    markers = markers + 1
    
    # Marcar a região desconhecida com zero
    markers[unknown == 255] = 0

    # Aplicar o algoritmo de watershed
    original_image = cv2.imread(img_path)
    markers = cv2.watershed(original_image, markers)
    original_image[markers == -1] = [255, 0, 0]
    
   
    # Intermediate images
    cv2.imwrite("{}cg-proc/img/edited/{}_opening.jpeg".format(master_path, name), opening)
    cv2.imwrite("{}cg-proc/img/edited/{}_sure_bg.jpeg".format(master_path, name), sure_bg)
    cv2.imwrite("{}cg-proc/img/edited/{}_sure_fg.jpeg".format(master_path, name), sure_fg)
    cv2.imwrite("{}cg-proc/img/edited/{}_unknown.jpeg".format(master_path, name), unknown)
    cv2.imwrite("{}cg-proc/img/edited/{}_markers.jpeg".format(master_path, name), markers.astype(np.uint8))
    cv2.imwrite("{}cg-proc/img/edited/{}_final.jpeg".format(master_path, name), original_image)
    

    # Função de seleção de círculo (HOUGH)
    circles = detect_circles(markers.astype(np.uint8))
    #circles = detect_circles(filled_image) # UTILIZADO PARA TESTES

    # Coordenadas utilizadas para o processamento geradas pelo Algoritmo de Hough
    #print("\nCircles: \n",circles,"\nFrom image: ",name)

    teste = cv2.imread("{}cg-proc/img/{}.jpeg".format(master_path, name.rstrip('_'))) # O _ porque não temos isso na imagem inicial
    
    # Desenha uma linha vermelha ao redor das moedas e adiciona na pasta circles
    draw_circles(teste, circles, "{}cg-proc/img/edited/final/circles/{}.jpeg".format(master_path, name))
